In [4]:
import pandas as pd
import numpy as np

In [5]:
url = 'https://drive.google.com/file/d/10V7NJujjslj5PwsBXVP8jwwAKizOJFCF/view?usp=sharing'
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
df = pd.read_csv(path)
df

,id,id_order,product_id,product_quantity,sku,unit_price,date
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38
...,...,...,...,...,...,...,...
293978,1650199,527398,0,1,JBL0122,42.99,2018-03-14 13:57:25
293979,1650200,527399,0,1,PAC0653,141.58,2018-03-14 13:57:34
293980,1650201,527400,0,2,APP0698,9.99,2018-03-14 13:57:41
293981,1650202,527388,0,1,BEZ0204,19.99,2018-03-14 13:58:01


In [6]:
test_df = df.copy()

In [7]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id                293983 non-null  int64 
 1   id_order          293983 non-null  int64 
 2   product_id        293983 non-null  int64 
 3   product_quantity  293983 non-null  int64 
 4   sku               293983 non-null  object
 5   unit_price        293983 non-null  object
 6   date              293983 non-null  object
dtypes: int64(4), object(3)
memory usage: 15.7+ MB


Changing Datatypes:

In [8]:
test_df['unit_price'] = pd.to_numeric(test_df['unit_price'])

ValueError: Unable to parse string "1.137.99" at position 6

In [9]:
test_df['unit_price'].str.count("\.").value_counts()

<>:1: SyntaxWarning: invalid escape sequence '\.'
<>:1: SyntaxWarning: invalid escape sequence '\.'
/tmp/ipython-input-2380186381.py:1: SyntaxWarning: invalid escape sequence '\.'
  test_df['unit_price'].str.count("\.").value_counts()


,count
unit_price,
1,257814
2,36169


In [ ]:
# Count the rows with more than one `.`
mult_decimal_rows = (test_df['unit_price'].str.count("\.")>1).sum()

# Find the percentage of corrupted rows
percent_corrupted = (100 * mult_decimal_rows / test_df.shape[0])
print(f"{percent_corrupted:.2f}% of the rows in our DataFrame have multiple decimal points in the unit_price")

Dropping 36,000 rows that has unit_price corrupted value

In [ ]:
# Boolean mask to find the orders that contain a price with multiple decimal points
multiple_decimal_mask = test_df['unit_price'].str.count("\.") > 1

# Apply the boolean mask to the orderlines DataFrame. This way we can find the order_id of all the affected orders.
corrupted_order_ids = test_df.loc[multiple_decimal_mask, "id_order"]

# Keep only the rows that do not have multiple decimal points
test_df = test_df.loc[~test_df['id_order'].isin(corrupted_order_ids)]

In [ ]:
test_df.shape[0]

In [ ]:
test_df["unit_price"] = pd.to_numeric(test_df["unit_price"])

In [ ]:
test_df['date'] = pd.to_datetime(test_df['date'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

In [ ]:
test_df.sample(10)

Dropping product_id column:

In [ ]:
test_df = test_df.drop(columns='product_id', axis=1)

In [ ]:
test_df['totalorder_price'] = (test_df['unit_price'] * test_df['product_quantity'])
test_df

In [ ]:
test_df['totalorder_price'] = pd.to_numeric(test_df['totalorder_price'], errors='coerce')

Dropping rows that has negative values:

In [ ]:
negative_unit_price = test_df.loc[test_df['unit_price'] < 0]

In [ ]:
test_df = test_df.drop(negative_unit_price.index)

Dropping 0.0 unit_price:

In [ ]:
zero_unit_price = test_df.loc[test_df['unit_price'] == 0, :]

In [ ]:
test_df

In [ ]:
test_df = test_df.drop(zero_unit_price.index)

Checking again for clean data:

In [ ]:
test_df.info()

In [ ]:
test_df.describe()

In [ ]:
test_df.duplicated(subset='id').sum()

In [ ]:
test_df.isna().sum()

In [ ]:
test_df.loc[df['product_quantity'] > 100,['id','product_quantity','sku']]

Top 10 Products in orders:

In [ ]:
test_df['sku'].value_counts().head(10)

Period of Range:

In [ ]:
test_df['date'].min()

In [ ]:
test_df['date'].max()

Min & Max Unit price:

In [ ]:
test_df['unit_price'].sort_values().head(20)

In [ ]:
test_df['unit_price'].max()

In [ ]:
order_totals = (
    test_df.groupby('id_order')['totalorder_price']
    .sum()
    .reset_index()
)
order_totals.sort_values(by='totalorder_price', ascending=False).head(20)

Top 50 orders:

In [ ]:
top50_orders = test_df[['id_order','sku','totalorder_price']].sort_values(by='totalorder_price', ascending=False).head(50)
print(top50_orders)

In [ ]:
top50_orders['sku'].value_counts()

Least 50 orders - Product list

In [ ]:
least50_orders = test_df[['id_order','sku','totalorder_price']].sort_values(by='totalorder_price', ascending=True).head(50)
least50_orders

In [ ]:
least50_orders['sku'].value_counts()

Revenue by year & month

In [ ]:
test_df['year_month'] = test_df['date'].dt.to_period('M')
test_df

In [ ]:
revenue_year = test_df.groupby('year_month')['totalorder_price'].sum()

In [ ]:
revenue_year

In [ ]:
revenue_orders_month = test_df.groupby('year_month').agg(
    total_revenue=('totalorder_price', 'sum'),
    total_orders=('id', 'nunique')).reset_index()   # use nunique to count unique orders

revenue_orders_month.rename(columns={'totalorder_price': 'total_revenue','id':'total_orderslist'}, inplace=True)
revenue_orders_month